In [ ]:
import matplotlib.pyplot as pltplt.figure(figsize=(10, 6))

all_log_times = state_logger.sample_times()
chosen_idxs = (6.65 < all_log_times) & (all_log_times < 6.8)

log_times = state_logger.sample_times()[chosen_idxs]
log_states = state_logger.data()[:, chosen_idxs]

#
for idx, r in enumerate(log_states[[10, 12]]):
    plt.plot(log_times, r, label=f"sim_joint_{idx+4}")
for idx, r in enumerate(log_states[[14, 15]]):
    plt.plot(log_times, 10 *r, label=f"sim_gripper_{idx}")
    
q_plan = np.hstack([q_traj.value(t) for t in log_times])
g_plan = np.hstack([g_traj.value(t) for t in log_times])
for idx, r in enumerate(q_plan[[3, 5]]):
    plt.plot(log_times, r, label=f"joint_{idx + 4}")
for idx, r in enumerate(g_plan):
    plt.plot(log_times, r, label=f"gripper_{idx}")
    
plt.axvline(
    x=gripper_cumulative_times_lst[-2],
    color="black", linestyle=(0, (1, 10)), label="t_release"
);
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1);
import numpy as np

import pydrake
from pydrake.all import (
    PiecewisePolynomial, RigidTransform, RotationMatrix
)

from src.drake_helpers import (
    BuildAndSimulateTrajectory, 
    setup_manipulation_station,
    visualize_transform, 
)
from src.ik import (
    create_q_knots, 
    pose_to_jointangles, 
    jointangles_to_pose, 
    spatial_velocity_jacobian_at_jointangles,
)
from src.throw import (
    plan_pickup, 
    plan_prethrow_pose,
    add_go_to_ja_via_jointinterpolation,
    add_go_to_pose_via_jointinterpolation,
    plan_throw,
)
from src.trajectory import get_launch_speed_required

# cleaner debugging
np.set_printoptions(precision=3, suppress=True)

In [ ]:
# start a single meshcat server instance to use for remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
print(web_url)

## The next cell determines everything relevant to the throw

In [ ]:
'''
setup
'''
# Get initial pose of the gripper by using default context of manip station.
P_WORLD_TARGET = np.array([-1, 1, 0])
GRIPPER_TO_OBJECT_DIST = 0.13 # meters
T_world_objectInitial = RigidTransform(
    p=[-.1, -.69, 1.04998503e-01],
    R=RotationMatrix.MakeZRotation(np.pi/2.0)
)
T_world_gripperObject = RigidTransform(
    p=T_world_objectInitial.translation() + np.array([0, 0, GRIPPER_TO_OBJECT_DIST]),
    R=RotationMatrix.MakeXRotation(-np.pi/2.0)
)
T_world_robotInitial, meshcat = setup_manipulation_station(T_world_objectInitial, zmq_url)

#object frame viz
visualize_transform(meshcat, "T_world_obj0", T_world_objectInitial)
T_world_target = RigidTransform(
    p=P_WORLD_TARGET,
    R=RotationMatrix()
)
visualize_transform(meshcat, "T_world_obj0", T_world_objectInitial)
visualize_transform(meshcat, "target", T_world_target)

'''
throw
* heading (top down view of the xy plane, relative to the positive x axis)
* release angle (relative to azimuth)
* release velocity
'''
print(f"-------Planning throw")
throw_heading = np.arctan2(P_WORLD_TARGET[1], P_WORLD_TARGET[0])

# we need these states to calculate the jacobian
ja1 = throw_heading - np.pi
# you can play around with these to achieve different "release" poses
prethrow_ja = np.array([ja1, 0, 0, 2.00, 0, -2.0, 0, 0, 0])
throwend_ja = np.array([ja1, 0, 0, 1.1, 0, -1.0, 0, 0, 0])
release_ja = prethrow_ja + 0.5 * (throwend_ja - prethrow_ja)
print(f"Release joint angles: {release_ja}")

# release ja => 6 dof pose => start position for projectile motion
T_world_release = jointangles_to_pose(release_ja[:7])

# estimate the throw angle by interpolating the joint poses a bit before and after release
before_ja = prethrow_ja + 0.499 * (throwend_ja - prethrow_ja)
after_ja = prethrow_ja + 0.501 * (throwend_ja - prethrow_ja)
T_world_before = jointangles_to_pose(before_ja[:7])
T_world_after = jointangles_to_pose(after_ja[:7])
p_world_before = T_world_before.translation()
p_world_after = T_world_after.translation()
delta = p_world_after - p_world_before
flat = np.copy(delta)
flat[2] = 0
angle_above_plane_rads = np.arccos(flat.dot(delta) / (np.linalg.norm(flat) * np.linalg.norm(delta)))
print(f"Throw angle (above xy plane) degs: {angle_above_plane_rads * 180.0 / np.pi}")

# calculate initial velocity magnitude
p_release = T_world_release.translation()
delta_x = np.linalg.norm(P_WORLD_TARGET[:2]) - np.linalg.norm(p_release[:2]) # dist on xy plane
delta_y = P_WORLD_TARGET[2] - p_release[2] # height difference
mag_v0_reqd = get_launch_speed_required(theta=angle_above_plane_rads, x=delta_x, y=delta_y)
print(f"Initial velocity magnitude: {mag_v0_reqd}")

V0 = mag_v0_reqd * np.array([
    np.sin(np.pi / 2 - angle_above_plane_rads) * np.cos(throw_heading),
    np.sin(np.pi / 2 - angle_above_plane_rads) * np.sin(throw_heading),
    np.cos(np.pi / 2 - angle_above_plane_rads),
])
V0 = np.expand_dims(V0, axis=1)

# calculate jacobian
J_G = spatial_velocity_jacobian_at_jointangles(release_ja[:7])
J_G_joint4 = J_G[3:, 3] # spatial velocity only
J_G_joint6 = J_G[3:, 5] # spatial velocity only
J_G_joints46 = np.zeros((3, 2))
J_G_joints46[:, 0] = J_G_joint4
J_G_joints46[:, 1] = J_G_joint6
print(J_G_joints46)

# find joint velocities (for 4 and 6) required to hit initial velocity knowing the jacobian
# Y = AX => X = (A_T * A)^-1 * A_T * Y, with Y = V0, A = J_G, X = dq
Y = V0
A = J_G_joints46
ATAinv = np.linalg.inv(A.T.dot(A))
X = ATAinv.dot(A.T.dot(Y))
joint_vels = X.squeeze()
print(f"joint velocities: {joint_vels}")

# calculate timing
dist = throwend_ja - prethrow_ja
time_reqd_j4 = dist[3] / joint_vels[0]
time_reqd_j6 = dist[5] / joint_vels[1]

print(f"time required j4: {time_reqd_j4}")
print(f"time required j6: {time_reqd_j6}")
assert(np.isclose(time_reqd_j4, time_reqd_j6, atol=1e-4)) # these times should be the same!
time_reqd = time_reqd_j6

'''
timings
'''
t_goToObj = 1.0
t_holdObj = 0.5
t_goToPreobj = 1.0
t_goToWaypoint = 2.0
t_goToPrethrow = 2.0 # must be greater than 1.0 for a 1 second hold to stabilize
t_goToThrowEnd = time_reqd

## This cell combines all the poses or joint angles with timings from above

In [ ]:
'''
go to poses
'''
# plan pickup
t_lst, q_knots, total_time = plan_pickup(T_world_robotInitial, T_world_gripperObject,
    t_goToObj=t_goToObj,
    t_holdObj=t_holdObj,
    t_goToPreobj=t_goToPreobj
)

# clear the bins via a waypoint
T_world_hackyWayPoint = RigidTransform(
    p=[-.6, -0.0, 0.6],
    R=RotationMatrix.MakeXRotation(-np.pi/2.0), #R_WORLD_PRETHROW, #RotationMatrix.MakeXRotation(-np.pi/2.0),
)
t_lst, q_knots = add_go_to_pose_via_jointinterpolation(
    T_world_robotInitial, 
    T_world_hackyWayPoint, 
    t_start=total_time,
    t_lst=t_lst,
    q_knots=q_knots,
    time_interval_s=t_goToWaypoint
)

# go to prethrow
t_lst, q_knots = add_go_to_ja_via_jointinterpolation(
    pose_to_jointangles(T_world_hackyWayPoint), 
    prethrow_ja, 
    t_start=total_time + t_goToWaypoint,
    t_lst=t_lst,
    q_knots=q_knots,
    time_interval_s=t_goToPrethrow,
    hold_time_s=1.0,
)

# go to throw
t_lst, q_knots = add_go_to_ja_via_jointinterpolation(
    prethrow_ja,
    throwend_ja,
    t_start=total_time + t_goToWaypoint + t_goToPrethrow,
    t_lst=t_lst,
    q_knots=q_knots,
    time_interval_s=t_goToThrowEnd,
    num_samples=5,
    include_end=True
)

# make gripper trajectory
delay_hack = 0.00
gripper_times_lst = np.array([
    0.,
    t_goToObj,
    t_holdObj,
    t_goToPreobj,
    t_goToWaypoint,
    t_goToPrethrow,
    t_goToThrowEnd / 2.0 + delay_hack, # should be some portion of t_goToThrowEnd
    t_goToThrowEnd / 2.0 - delay_hack, # should be some portion of t_goToThrowEnd
])
gripper_cumulative_times_lst = np.cumsum(gripper_times_lst)
GRIPPER_OPEN = 0.5
GRIPPER_CLOSED = 0.0
gripper_knots = np.array([
    GRIPPER_OPEN,
    GRIPPER_OPEN,
    GRIPPER_CLOSED,
    GRIPPER_CLOSED,
    GRIPPER_CLOSED,
    GRIPPER_CLOSED,
    GRIPPER_CLOSED,
    GRIPPER_OPEN,
]).reshape(1,gripper_times_lst.shape[0])
g_traj = PiecewisePolynomial.FirstOrderHold(gripper_cumulative_times_lst, gripper_knots)

In [ ]:
# turn trajectory into joint space
q_knots = np.array(q_knots)
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:7].T)

In [ ]:
# do the thing
simulator, station_plant, meshcat, state_logger = BuildAndSimulateTrajectory(q_traj, g_traj, T_world_objectInitial, None, zmq_url)
visualize_transform(
    meshcat,
    "TARGET",
    RigidTransform(RotationMatrix.MakeZRotation(0), P_WORLD_TARGET),
    prefix='',
    length=0.3,
    radius=0.02
)

throw_time = 2
meshcat.start_recording()
print(f"Throw motion should happen from 6.5 seconds to {6.5 + time_reqd} seconds")
print(f"Running for {q_traj.end_time() + throw_time} seconds")
simulator.AdvanceTo(q_traj.end_time() + throw_time)
meshcat.stop_recording()
meshcat.publish_recording()

## Simulation Viz + Comparison with Plan

In [ ]:
plt.figure(figsize=(10, 6))

all_log_times = state_logger.sample_times()
chosen_idxs = (6.45 < all_log_times) & (all_log_times < 6.8)

log_times = state_logger.sample_times()[chosen_idxs]
log_states = state_logger.data()[:, chosen_idxs]

#
for idx, r in enumerate(log_states[[10, 12]]):
    plt.plot(log_times, r, label=f"sim_joint_{2 * idx+4}")
for idx, r in enumerate(log_states[[14, 15]]):
    plt.plot(log_times, 10 *r, label=f"sim_gripper_{idx}")
    
q_plan = np.hstack([q_traj.value(t) for t in log_times])
g_plan = np.hstack([g_traj.value(t) for t in log_times])
for idx, r in enumerate(q_plan[[3, 5]]):
    plt.plot(log_times, r, label=f"joint_{2 * idx + 4}")
for idx, r in enumerate(g_plan):
    plt.plot(log_times, r, label=f"gripper_{idx}")
    
plt.axvline(
    x=gripper_cumulative_times_lst[-2],
    color="black", linestyle=(0, (1, 10)), label="t_release"
);
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1);